In [26]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import shutil

# Ensure ~/.kaggle directory exists
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# Move kaggle.json to the correct location
shutil.move("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))

# Change file permissions to avoid errors
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 600)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\LAPTOP/.kaggle/kaggle.json'

In [5]:
!kaggle datasets download -d purusinghvi/email-spam-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/purusinghvi/email-spam-classification-dataset
License(s): MIT


In [6]:
import os
print(os.listdir())  # Lists all files in the current directory


['2025 fresher hiring_Student brochure.pdf', '2560x1440-500871-wallpaper-computer.jpg', '2560x1440-521782-hd-nature-wallpaper.jpg', '3126F897-D508-4C0B-ADD2-A9FAD6A39694', '3rd Sem backlog reciept.pdf', '4k wallpapers (1).jpeg', '4k wallpapers (2).jpeg', '4k wallpapers.jpeg', '66_updated_AI & ML LAB.pdf', '6th SEM Exam Fees.pdf', '6th SEM FEES RECIEPT.pdf', '7th Sem Enrollment form.pdf', '7th SEM EXAM FEES RECIEPT.pdf', '8K Spiderman live wallpaper.jpeg', '8th SEM EXAM FEES RECIEPT.pdf', 'Accessing pandas dataframe columns.docx', 'Admit card (1).pdf', 'Admit card (2).pdf', 'Admit card.pdf', 'AI & ML LAB Copy.docx', 'AI & ML LAB New.docx', 'AI & ML LAB_compressed.docx', 'AICTE Internship Offer Letter.pdf', 'alex-knight-wfwUpfVqrKU-unsplash.jpg', "Alzheimer's Disease Detection Chatbot PPT.pptx", "Alzheimer's Disease Detection Chatbot.ipynb", 'Alzheimers_Flowchart_Updated.png', 'An Introduction to Machine Learning doc.pptx', 'Anaconda3-2022.10-Windows-x86_64 (1).exe', 'Anaconda3-2022.10-W

In [7]:
import zipfile

with zipfile.ZipFile("email-spam-classification-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")  # Extracts files into the "data" folder


In [8]:
print(os.listdir("data"))  # Lists extracted files


['combined_data.csv']


In [9]:
import pandas as pd
data = pd.read_csv("data/combined_data.csv")  # Adjust filename as needed
print(data.head())

   label                                               text
0      1  ounce feather bowl hummingbird opec moment ala...
1      1  wulvob get your medircations online qnb ikud v...
2      0   computer connection from cnn com wednesday es...
3      1  university degree obtain a prosperous future m...
4      0  thanks for all your answers guys i know i shou...


In [10]:
print(data.columns)

Index(['label', 'text'], dtype='object')


In [11]:
print(data.head(),"\n\n")  # Display first few rows
print(data.columns,"\n\n")  # Show column names
print(data.info())  # Check data types

   label                                               text
0      1  ounce feather bowl hummingbird opec moment ala...
1      1  wulvob get your medircations online qnb ikud v...
2      0   computer connection from cnn com wednesday es...
3      1  university degree obtain a prosperous future m...
4      0  thanks for all your answers guys i know i shou... 


Index(['label', 'text'], dtype='object') 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83448 entries, 0 to 83447
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   83448 non-null  int64 
 1   text    83448 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB
None


In [12]:
print(data.isnull().sum())  # Shows the count of missing values per column

label    0
text     0
dtype: int64


In [13]:
print(data[data.isnull().any(axis=1)])  # Displays rows with at least one missing value

Empty DataFrame
Columns: [label, text]
Index: []


In [14]:
import re

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

# Apply cleaning function to dataset
data['cleaned_text'] = data['text'].apply(clean_text)

# Check the first few cleaned entries
print(data[['text', 'cleaned_text']].head())

                                                text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                        cleaned_text  
0  ounce feather bowl hummingbird opec moment ala...  
1  wulvob get your medircations online qnb ikud v...  
2   computer connection from cnn com wednesday es...  
3  university degree obtain a prosperous future m...  
4  thanks for all your answers guys i know i shou...  


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Convert text into numerical features
X = vectorizer.fit_transform(data['cleaned_text'])  # Features
y = data['label']  # Target labels (spam or ham)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [18]:
model = MultinomialNB()
model.fit(X_train, y_train)  # Train the classifier

MultinomialNB()

In [19]:
predictions = model.predict(X_test)

In [20]:
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.9768723786698622


In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      7938
           1       0.99      0.97      0.98      8752

    accuracy                           0.98     16690
   macro avg       0.98      0.98      0.98     16690
weighted avg       0.98      0.98      0.98     16690



In [22]:
new_emails = ["Congratulations! You've won a free gift, claim now.", 
              "Hey, want to grab lunch tomorrow?",
              "Limited-time offer! Buy now and get 50% off.",
              "Hello! Purchase our courses for cheap!"]

new_vectorized = vectorizer.transform(new_emails)  # Convert emails to numerical format
print(model.predict(new_vectorized))  # Outputs: 1 for spam, 0 for ham


[1 0 1 1]


In [23]:
import pickle

# Save the trained model
with open("spam_classifier.pkl", "wb") as f:
    pickle.dump(model, f)

# Save the TF-IDF vectorizer too (since text features need conversion)
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [24]:
#RELOAD THE MODEL WITHOUT TRAINING TO AVOID OVERFITTING

import pickle

# Load the model
with open("spam_classifier.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Load the vectorizer
with open("vectorizer.pkl", "rb") as f:
    loaded_vectorizer = pickle.load(f)


In [25]:
#TEST THE MODEL TO ENSURE ITS WORKING PROPERLY


test_email = ["You’ve won a lottery! Claim your prize now.","Offers on Pizza!"]
test_vectorized = loaded_vectorizer.transform(test_email)

print(loaded_model.predict(test_vectorized))  # Outputs 1 (spam) or 0 (ham)


[1 1]
